In [1]:
import argparse
import pathlib
import shutil
import sys

import numpy as np
import tqdm

try:
    cfg = get_ipython().config
    in_notebook = True
except NameError:
    in_notebook = False
    # check if in a jupyter notebook
try:
    cfg = get_ipython().config
    in_notebook = True
except NameError:
    in_notebook = False

# Get the current working directory
cwd = pathlib.Path.cwd()

if (cwd / ".git").is_dir():
    root_dir = cwd

else:
    root_dir = None
    for parent in cwd.parents:
        if (parent / ".git").is_dir():
            root_dir = parent
            break

# Check if a Git root directory was found
if root_dir is None:
    raise FileNotFoundError("No Git root directory found.")

sys.path.append(str(pathlib.Path(f"{root_dir}/utils").resolve()))
from file_checking import check_number_of_files

In [2]:
if not in_notebook:
    argparser = argparse.ArgumentParser(
        description="set up directories for the analysis of the data"
    )

    argparser.add_argument(
        "--patient",
        type=str,
        required=True,
        help="patient name, e.g. 'P01'",
    )
    argparser.add_argument(
        "--well_fov",
        type=str,
        help="Path to the input directory containing the tiff images",
    )

    args = argparser.parse_args()
    patient = args.patient
    well_fov = args.well_fov
else:
    patient = "NF0014"
    well_fov = "C10-1"

In [3]:
# set path to the processed data dir
segmentation_data_dir = pathlib.Path(
    f"{root_dir}/data/{patient}/segmentation_masks/{well_fov}"
).resolve(strict=True)
zstack_dir = pathlib.Path(
    f"{root_dir}/data/{patient}/zstack_images/{well_fov}"
).resolve(strict=True)

In [4]:
# perform checks for each directory
segmentation_data_files = list(segmentation_data_dir.glob("*"))
segmentation_data_files

[PosixPath('/home/lippincm/Documents/GFF_3D_organoid_profiling_pipeline/data/NF0014/segmentation_masks/C10-1/organoid_masks_reconstructed.tiff'),
 PosixPath('/home/lippincm/Documents/GFF_3D_organoid_profiling_pipeline/data/NF0014/segmentation_masks/C10-1/nuclei_reconstruction_dict.npy'),
 PosixPath('/home/lippincm/Documents/GFF_3D_organoid_profiling_pipeline/data/NF0014/segmentation_masks/C10-1/organoid_masks_reconstructed_corrected.tiff'),
 PosixPath('/home/lippincm/Documents/GFF_3D_organoid_profiling_pipeline/data/NF0014/segmentation_masks/C10-1/nuclei_masks_reassigned.tiff'),
 PosixPath('/home/lippincm/Documents/GFF_3D_organoid_profiling_pipeline/data/NF0014/segmentation_masks/C10-1/organoid_masks_decoupled.tiff'),
 PosixPath('/home/lippincm/Documents/GFF_3D_organoid_profiling_pipeline/data/NF0014/segmentation_masks/C10-1/cytoplasm_mask.tiff'),
 PosixPath('/home/lippincm/Documents/GFF_3D_organoid_profiling_pipeline/data/NF0014/segmentation_masks/C10-1/nuclei_masks_decoupled.tiff'),


## Copy files from processed dir to cellprofiler images dir

In [5]:
masks_names_to_keep = [
    "cell_masks_watershed.tiff",
    "cytoplasm_mask.tiff",
    "nuclei_masks_reassigned.tiff",
    "organoid_masks_reconstructed.tiff",
]

In [6]:
# remove files that are not in the list of masks to keep
for file in tqdm.tqdm(segmentation_data_files):
    if file.name not in masks_names_to_keep:
        file.unlink()
        print(f"Removed file: {file.name}")

100%|██████████| 12/12 [00:00<00:00, 3095.62it/s]

Removed file: nuclei_reconstruction_dict.npy
Removed file: organoid_masks_reconstructed_corrected.tiff
Removed file: organoid_masks_decoupled.tiff
Removed file: nuclei_masks_decoupled.tiff
Removed file: cell_masks_corrected.tiff
Removed file: organoid_reconstruction_dict.npy
Removed file: nuclei_masks_reconstructed.tiff
Removed file: nuclei_masks_reconstructed_corrected.tiff


In [7]:
# copy the masks to the zstack directory
for file in tqdm.tqdm(segmentation_data_files):
    if file.name in masks_names_to_keep:
        destination = zstack_dir / file.name
        if not destination.exists():
            shutil.copy(file, destination)
            print(f"Copied file: {file.name} to {destination}")
        else:
            print(f"File {file.name} already exists in {zstack_dir}, skipping copy.")

  8%|▊         | 1/12 [00:00<00:01,  8.45it/s]

Copied file: organoid_masks_reconstructed.tiff to /home/lippincm/Documents/GFF_3D_organoid_profiling_pipeline/data/NF0014/zstack_images/C10-1/organoid_masks_reconstructed.tiff


 33%|███▎      | 4/12 [00:00<00:00, 18.73it/s]

Copied file: nuclei_masks_reassigned.tiff to /home/lippincm/Documents/GFF_3D_organoid_profiling_pipeline/data/NF0014/zstack_images/C10-1/nuclei_masks_reassigned.tiff


 50%|█████     | 6/12 [00:00<00:00, 18.03it/s]

Copied file: cytoplasm_mask.tiff to /home/lippincm/Documents/GFF_3D_organoid_profiling_pipeline/data/NF0014/zstack_images/C10-1/cytoplasm_mask.tiff


100%|██████████| 12/12 [00:00<00:00, 26.43it/s]

Copied file: cell_masks_watershed.tiff to /home/lippincm/Documents/GFF_3D_organoid_profiling_pipeline/data/NF0014/zstack_images/C10-1/cell_masks_watershed.tiff
